*Copyright (C) 2022-23 Intel Corporation*<br>
*SPDX-License-Identifier: BSD-3-Clause*<br>
*See: https://spdx.org/licenses/*

---

# Tutorial 2: Elementwise products

**Motivation:** In this tutorial, we will highlight more of the standard library included with Lava-VA. Here we demonstrate the element-wise product of vectors using ProductVec.




First, we make the imports and connect to Loihi 2.

In [1]:
from pylab import *

In [2]:
import lava.frameworks.loihi2 as lv

In [3]:
from lava.utils import loihi

loihi.use_slurm_host(loihi_gen=loihi.ChipGeneration.N3B3)
use_loihi2 = loihi.is_installed()

if use_loihi2:
    run_cfg = lv.Loihi2HwCfg()
    print("Running on Loihi 2")
else:
    run_cfg = lv.Loihi2SimCfg(select_tag='fixed_pt')
    print("Loihi2 compiler is not available in this system. "
          "This tutorial will execute on CPU backend.")

Running on Loihi 2


Next, we will setup the inputs and initialize the input weights.

In [4]:
num_steps = 10
weights1 = np.zeros((5,1))
weights2 = np.zeros((5,1))

weights1[:,0] = [2, 6, 10, -2, -6]
weights2[:,0] = [4, 8, 12, -4, 8]

weights1 /= 16
weights2 /= 16

inp_shape = (weights1.shape[1],)
out_shape = (weights1.shape[0],)

inp_data = np.zeros((inp_shape[0], num_steps))
inp_data[:, 2] = 16
inp_data[:, 6] = 32

Then we instantiate the objects in the network.

In [5]:
dense1 = lv.GradedDense(weights=weights1)
dense2 = lv.GradedDense(weights=weights2)

vec = lv.ProductVec(shape=out_shape, vth=1, exp=0)

generator1 = lv.InputVec(inp_data, loihi2=use_loihi2)
generator2 = lv.InputVec(inp_data, loihi2=use_loihi2)
monitor = lv.OutputVec(shape=out_shape, buffer=num_steps,
                       loihi2=True)

In this case, ProductVec is an object that has two input channels. We can access those input channels by concatenating the objects and "piping" them into the ProductVec layer. 

In [6]:
vec << (dense1 @ generator1, dense2 @ generator2)
monitor << vec

In [7]:
try:
    vec.run(condition=lv.RunSteps(num_steps=num_steps),
              run_cfg=run_cfg)
    out_data = monitor.get_data()
finally:
    vec.stop()

In [8]:
out_data[:, (3,7)]

array([[   8,   32],
       [  48,  192],
       [ 120,  480],
       [   8,   32],
       [ -48, -192]], dtype=int32)

In [9]:
(weights1 @ inp_data[:,2]) * (weights2 @ inp_data[:,2])

array([  8.,  48., 120.,   8., -48.])

We can see that this matches the numpy calculation.

## Multiplication operator overload

Similar to addition, the multiplication operator is overloaded inside of GradedVec to enable the use of algebraic syntax to compute the elementwise product.


In [10]:
dense1 = lv.GradedDense(weights=weights1)
dense2 = lv.GradedDense(weights=weights2)

vec1 = lv.GradedVec(shape=out_shape, vth=1, exp=0)
vec2 = lv.GradedVec(shape=out_shape, vth=1, exp=0)

outvec = lv.GradedVec(shape=out_shape, vth=1, exp=0)

generator1 = lv.InputVec(inp_data, loihi2=use_loihi2)
generator2 = lv.InputVec(inp_data, loihi2=use_loihi2)
monitor = lv.OutputVec(shape=out_shape, buffer=num_steps,
                       loihi2=True)


In [11]:
vec1 << dense1 @ generator1
vec2 << dense2 @ generator2
outvec << vec1 * vec2
monitor << outvec

In [12]:
try:
    vec1.run(condition=lv.RunSteps(num_steps=num_steps),
              run_cfg=run_cfg)
    out_data = monitor.get_data()
finally:
    vec1.stop()

In [13]:
out_data[:, (5,)] 

array([[  8],
       [ 48],
       [120],
       [  8],
       [-48]], dtype=int32)